In [8]:
import pandas as pd
from google.cloud import storage
from datetime import date
from tiingo import TiingoClient
import time

def prepare_csv():
    #Tiingo API Config
    config = {}
    config['session'] = True
    config['api_key'] = "0015ea3a0ed951cea8f45258393fd6b595327627"
    client = TiingoClient(config)
    
    #Pull BTC data into dictionary
    historical_prices = client.get_crypto_price_history(tickers = ['BTCUSD'], startDate='2019-12-31', endDate='2022-12-31', resampleFreq='24Hour')
    
    #Convert to pandas dataframe
    df = pd.DataFrame(historical_prices[0]["priceData"])
    df["ticker"] = "BTC"
    df['tradesDone'] = df['tradesDone'].astype(int)
    return df

def load_csv(df, ticker):
    #GCS Config
    client = storage.Client()
    bucket = client.get_bucket('data_lake_stocks-data-pipeline')
    #Load dataframe to GCS as .csv
    bucket.blob(f'{ticker} Three Year Daily Price History').upload_from_string(df.to_csv(), "BTC Three Year Daily Price History")
    
def main():
    #Create dataframe with BTC data
    csv = prepare_csv()
    #load data to GCS
    load_csv(csv, "BTC")

if __name__ == "__main__":
    main()



,date,open,high,low,close,volume,volumeNotional,tradesDone,ticker
0,2019-12-31T00:00:00+00:00,7240.999174,7316.345364,7138.917937,7189.816307,37159.587553,2.671706e+08,323932,BTC
1,2020-01-01T00:00:00+00:00,7190.600511,7254.002643,7168.967989,7196.776430,23489.516384,1.690488e+08,244774,BTC
2,2020-01-02T00:00:00+00:00,7196.346029,7208.283064,6923.563861,6963.684559,46760.010923,3.256220e+08,370066,BTC
3,2020-01-03T00:00:00+00:00,6963.407320,7407.114205,6870.723993,7345.355821,92083.873024,6.763888e+08,628418,BTC
4,2020-01-04T00:00:00+00:00,7346.201139,7404.769841,7273.229248,7354.871466,40894.009749,3.007702e+08,360486,BTC
...,...,...,...,...,...,...,...,...,...
1092,2022-12-27T00:00:00+00:00,16917.800962,16970.812742,16587.583745,16700.907708,197051.180338,3.290934e+09,4392883,BTC
1093,2022-12-28T00:00:00+00:00,16701.122888,16778.259650,16458.928683,16540.712866,224381.399187,3.711428e+09,4925311,BTC
1094,2022-12-29T00:00:00+00:00,16540.631322,16658.824721,16482.148419,16628.405212,185563.453849,3.085624e+09,4096487,BTC
1095,2022-12-30T00:00:00+00:00,16628.519658,16668.651620,16327.342551,16601.265248,198631.892141,3.297541e+09,3983509,BTC
